In [1]:
import gensim
import sklearn
from sklearn.externals import joblib
from sklearn import svm

In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format('vectors/GoogleNews-vectors-negative300.bin', binary=True)

In [4]:
model.wv.similarity('woman', 'man')




0.76640122309953518

In [5]:
from sklearn.externals import joblib
import pandas as pd
from sklearn.metrics import accuracy_score

clf = joblib.load('models/svm.pkl') 

test_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_test_vectorized.tsv', sep='\t', header=None)
val_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_val_vectorized.tsv', sep='\t', header=None)

# Test and validation in one go bc i'm lazy
for test_name, test_df in zip(['test', 'val'], [test_vectorized, val_vectorized]):
	orig_rows, orig_cols = test_df.shape

	# Remove rows with NaN
	test_df.dropna(axis=0, inplace=True)

	# Count number of rows removed
	diff = orig_rows - test_df.shape[0]

	X = test_df.iloc[:, :-1]
	y = test_df.iloc[:, -1]

	preds = clf.predict(X)

	num_correct = accuracy_score(y, preds, normalize=False)

	print test_name, ": percentage non-nan correct:", num_correct/float(test_df.shape[0]) 
	print test_name, ": percentage correct overall", num_correct/float(orig_rows)
    

test : percentage non-nan correct: 0.923381396664
test : percentage correct overall 0.894794520548
val : percentage non-nan correct: 0.931343283582
val : percentage correct overall 0.871508379888


In [12]:
train = pd.read_csv('lexical_entailment/bless2011/data_lex_train_vectorized_asym.tsv', sep='\t', header=None)
train.dropna(axis=0, inplace=True)
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

clf = svm.LinearSVC(class_weight='balanced')
clf.fit(X, y)

LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [22]:
clf.coef_

array([[-0.49308603,  0.34003836,  0.46912924,  0.11621957,  1.23653491,
        -0.38403792, -1.01614184, -0.07808303, -0.16723804,  1.03401088,
         0.20314171,  1.10102852,  0.01577937, -0.10991365, -0.03696456,
         0.42136979,  0.09331233,  0.57645631, -0.56390412, -0.170867  ,
        -0.00471364,  1.06828664,  0.63652294, -0.43181467, -0.64496962,
         0.1871348 ,  0.18631261, -0.08315926,  0.42526042,  0.36943712,
         0.62032774,  0.79282776, -0.02282934, -0.28663304, -0.1269898 ,
        -0.76709666,  0.68522672,  0.98283319,  0.34055238, -0.2578815 ,
        -0.24615537, -0.84236196,  0.73760476,  0.07406205, -0.1077655 ,
        -0.06704011,  0.29467109, -0.57944896, -0.11359085,  0.5628705 ,
        -0.48747097,  0.74891291, -0.08657377,  0.35194516,  0.27134764,
        -0.18206729, -1.25931868, -0.28736611,  0.04231769, -0.14089536,
        -0.45065302,  1.00489075, -0.00928573, -0.0936773 ,  1.14136614,
        -0.1609479 ,  0.15626993,  0.21002019, -0.4

In [25]:
test_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_test_vectorized_asym.tsv', sep='\t', header=None)
test = pd.read_csv('lexical_entailment/bless2011/data_lex_test.tsv', sep='\t', header=None)
val_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_val_vectorized_asym.tsv', sep='\t', header=None)

In [38]:
test_df = val_vectorized
test_df.dropna(axis=0, inplace=True)
X = test_df.iloc[:, :-1]
y = test_df.iloc[:, -1]

In [42]:
preds = clf.predict(X)

In [43]:
print "precision", metrics.precision_score(y, preds)
print "recall", metrics.recall_score(y, preds)
print "f1", metrics.f1_score(y, preds)
print "True", metrics.accuracy_score(y[y == 1], preds[y == 1])
print "False", metrics.accuracy_score(y[y == 0], preds[y == 0])

precision 0.806451612903
recall 0.892857142857
f1 0.847457627119
True 0.892857142857
False 0.980456026059
